In [1]:
import numpy as np
import pandas as pd
import sqlite3

In [2]:
df_ratings = pd.read_csv('df_interactions.csv',index_col=0)
df_ratings = df_ratings.groupby(['id','name','username']).mean().reset_index()
df_games = pd.read_csv('boardgames_ranks.csv')
df_users = df_ratings[['username','name','rating']].groupby('username').agg({'name':'nunique'}).reset_index()

In [3]:
df_extra = pd.read_csv('games_data.csv')

In [4]:
df_games = df_games.merge(df_extra[['id','description','image_name']],how='left', left_on='id', right_on='id')

In [5]:
df_games

,id,name,yearpublished,rank,bayesaverage,average,usersrated,abstracts_rank,cgs_rank,childrensgames_rank,familygames_rank,partygames_rank,strategygames_rank,thematic_rank,wargames_rank,description,image_name
0,224517,Brass: Birmingham,2018,1,8.41581,8.59842,44835,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,"Build networks, grow industries, and navigate ...",224517.jpg
1,161936,Pandemic Legacy: Season 1,2015,2,8.38155,8.52941,53143,NaN,NaN,NaN,NaN,NaN,2.0,1.0,NaN,Mutating diseases are spreading around the wor...,161936.jpg
2,174430,Gloomhaven,2017,3,8.35758,8.59323,61748,NaN,NaN,NaN,NaN,NaN,4.0,2.0,NaN,Vanquish monsters with strategic cardplay. Ful...,174430.jpg
3,342942,Ark Nova,2021,4,8.33266,8.53712,42099,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,"Plan and build a modern, scientifically manage...",342942.jpg
4,233078,Twilight Imperium: Fourth Edition,2017,5,8.24384,8.60632,23407,NaN,NaN,NaN,NaN,NaN,5.0,3.0,NaN,"Build an intergalactic empire through trade, r...",233078.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153981,420272,Dream Games – Kapitel 1: Böses Erwachen,2023,0,0.00000,8.00000,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153982,420273,Vermisst in Vorarlberg,2021,0,0.00000,0.00000,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153983,420274,Cold Case Detectives – Fall #1: Tödlicher Somm...,2022,0,0.00000,7.00000,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153984,420275,Turret Dice: The Defenders,0,0,0.00000,10.00000,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
def determine_genres(row):
    genre_categories = [
        'abstracts_rank', 'cgs_rank', 'childrensgames_rank', 
        'familygames_rank', 'partygames_rank', 'strategygames_rank', 
        'thematic_rank', 'wargames_rank'
    ]
    genre_names = [
        'Abstracts', 'CGS', 'ChildrensGames', 
        'FamilyGames', 'PartyGames', 'StrategyGames', 
        'Thematic', 'WarGames'
    ]
    genres = []
    for category, genre in zip(genre_categories, genre_names):
        if pd.notnull(row[category]):
            genres.append(genre)
    return ', '.join(genres) if genres else 'Unknown'

df_games['genre'] = df_games.apply(determine_genres, axis=1)

In [7]:
df_games = df_games.loc[0:1999]

In [8]:
# Создание соединения с базой данных SQLite
conn = sqlite3.connect('bgg_2000.db')

# Создание таблицы games с правильными типами данных и ограничениями
df_games.to_sql('games', conn, if_exists='replace', index=False, 
                dtype={
                    'id': 'INTEGER PRIMARY KEY',
                    'name': 'TEXT'
                })

# Создание таблицы users
df_users.to_sql('users', conn, if_exists='replace', index=False, 
                dtype={
                    'username': 'TEXT PRIMARY KEY',
                    'name': 'TEXT'
                })

# Создание таблицы ratings с уникальным ограничением
df_ratings.sample(frac=0.3, random_state=1).to_sql('ratings', conn, if_exists='replace', index=False, 
                                              dtype={
                                                  'id': 'INTEGER',
                                                  'username': 'TEXT',
                                                  'rating': 'INTEGER'
                                              })

# Закрытие соединения
conn.close()

# Добавление уникального ограничения на таблицу ratings
with sqlite3.connect('bgg_2000.db') as conn:
    cursor = conn.cursor()
    cursor.execute('''
        CREATE UNIQUE INDEX IF NOT EXISTS idx_game_user 
        ON ratings (id, username)
    ''')
    conn.commit()